In [ ]:
using Pandas ;
using Plots ;
using CSV ;
using RollingFunctions;

In [ ]:
real_data_dir = "../COVID-19/csse_covid_19_data/csse_covid_19_time_series"
real_confirmed = joinpath(real_data_dir,"time_series_covid19_confirmed_US.csv")
real_deaths = joinpath(real_data_dir,"time_series_covid19_deaths_US.csv")

In [ ]:
df_suffolk_confirmed = read_csv(real_confirmed)
suffolk_infected = iloc(query(df_suffolk_confirmed,:(Admin2 == "Miami-Dade" && Province_State == "Florida")))[:,12:size(df_suffolk_confirmed)[2]]

In [ ]:
# NOTE - this table has "Population" at column 12
df_suffolk_deaths = read_csv(real_deaths)
suffolk_deaths = iloc(query(df_suffolk_deaths,:(Admin2 == "Miami-Dade" && Province_State == "Florida")))[:,13:size(df_suffolk_deaths)[2]]

In [ ]:
function get_data(df)
    cumulative_counts::Array{Int64, 1} = []
    for x in sum(df)
        push!(cumulative_counts, convert(Int, x))    
    end

    inf1 = cumulative_counts[2:end]
    inf2 = cumulative_counts[1:end-1]
    per_day_counts = inf1 - inf2
    return cumulative_counts, per_day_counts
end

inf, delta_inf = get_data(suffolk_infected)
deaths, delta_deaths = get_data(suffolk_deaths)

In [ ]:
Plots.plot(delta_inf[70:end])

In [ ]:
window = 7
@assert window % 2 != 0
cut = Int(floor(window/2)) # on each side
smooth_delta_inf = rollmean(delta_inf, window)
smooth_inf = rollmean(inf, window)
Plots.plot(hcat(delta_inf[1+cut:end-cut], smooth_delta_inf), label=["before" "after"], legend=:topleft)

In [ ]:
pop = 2716940
N = findfirst(map(x-> x >= 0.0025 * pop, inf)) - 7

In [ ]:
Plots.plot(hcat(delta_inf[N+cut:N+cut+163], smooth_delta_inf[N:N+163]), label=["before" "after"], legend=:topleft)

In [ ]:
println(inf[70:end] ./ 10000000)

In [ ]:
println(sum(smooth_delta_inf))
println(inf[end])

In [ ]:
Plots.plot(hcat(
        convert(Array{Any}, cumsum(smooth_delta_inf)), # TODO - can we avoid this convert?
        inf[1+cut+1:end-cut]), label=["smooth" "reg"], legend=:topleft)

In [ ]:
println(size(inf))
println(size(deaths))

In [ ]:
# Want to start our trajectories from 5% infected (relative to total eventual infected)
N = findfirst(map(x-> x >= 0.05 * inf[end], inf))

In [ ]:
# Consider only a 100 day trajectory

all_1 = Plots.plot(hcat(inf, deaths), label=["inf" "deaths"], legend=nothing)
cumulative = Plots.plot(hcat(inf[N:N+100], deaths[N:N+100]), label=["inf" "deaths"], legend=nothing)
all_2 = Plots.plot(hcat(delta_inf, delta_deaths), label=["inf" "deaths"], legend=nothing)
deltas = Plots.plot(hcat(delta_inf[N:N+100], delta_deaths[N:N+100]), label=["inf" "deaths"], legend=nothing)
Plots.plot(all_1, cumulative, all_2, deltas)

In [ ]:
deaths[end]/inf[end]

In [ ]:
pop = 803901
nodes = 3000
mort_rate = 0.0486
println(inf[end] / pop * nodes )
println(inf[end] * 10 / pop * nodes)

println(deaths[end] / mort_rate / pop * nodes)
println(deaths[end] / mort_rate * 10 / pop * nodes)